## Chapter 11 - Training Deep Neural Networks

In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split

loading CIFAR10 dataset and splitting it.

In [2]:
(X_train , y_train) , (X_test , y_test) = keras.datasets.cifar10.load_data()

In [3]:
X_train , X_val , y_train , y_val = train_test_split(X_train , y_train , train_size=40000 , random_state=42)
print(X_train.shape , X_val.shape , X_test.shape)

(40000, 32, 32, 3) (10000, 32, 32, 3) (10000, 32, 32, 3)


building DNN with 20 hidden layers of 100 neurons at each layer, using He initialization, ELU activation and nadam optimizer.

In [4]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = (32,32,3))
])
for layer in range(20):
    model.add(keras.layers.Dense(100 , kernel_initializer = "he_normal" ,
                                activation = 'elu'))

model.add(keras.layers.Dense(10 , activation = "softmax"))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               307300    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               1

In [6]:
early_stop = keras.callbacks.EarlyStopping(patience=5)
model.compile(loss = "sparse_categorical_crossentropy" , optimizer = "nadam" , metrics=["accuracy"])

training the DNN with early stopping, we got 1.6636 val_loss and 0.3944 accuracy on the validation set after 26 epochs.

In [7]:
history = model.fit(X_train , y_train , validation_data=(X_val, y_val) , callbacks=[early_stop] ,
                    epochs = 50 , batch_size = 64)

Epoch 1/50
625/625 [==============================] - 4s 7ms/step - loss: 5.0356 - accuracy: 0.1984 - val_loss: 1.9532 - val_accuracy: 0.2793
Epoch 2/50
625/625 [==============================] - 4s 7ms/step - loss: 1.9701 - accuracy: 0.2797 - val_loss: 1.9612 - val_accuracy: 0.2904
Epoch 3/50
625/625 [==============================] - 4s 7ms/step - loss: 1.8991 - accuracy: 0.3043 - val_loss: 1.8455 - val_accuracy: 0.3322
Epoch 4/50
625/625 [==============================] - 4s 7ms/step - loss: 1.8611 - accuracy: 0.3211 - val_loss: 1.9038 - val_accuracy: 0.2934
Epoch 5/50
625/625 [==============================] - 4s 7ms/step - loss: 1.8395 - accuracy: 0.3319 - val_loss: 1.8445 - val_accuracy: 0.3246
Epoch 6/50
625/625 [==============================] - 5s 7ms/step - loss: 1.8280 - accuracy: 0.3343 - val_loss: 1.8000 - val_accuracy: 0.3417
Epoch 7/50
625/625 [==============================] - 4s 7ms/step - loss: 1.8163 - accuracy: 0.3404 - val_loss: 1.8223 - val_accuracy: 0.3428
Epoch 

training the DNN model but this time adding batch normalization after each Dense layer and the Input layer, before activation layer.

In [12]:
model2 = keras.models.Sequential([
    keras.layers.Flatten(input_shape = (32,32,3)) ,
    keras.layers.BatchNormalization()
])
for layer in range(20):
    model2.add(keras.layers.Dense(100 , kernel_initializer = "he_normal"))
    model2.add(keras.layers.BatchNormalization())
    model2.add(keras.layers.Activation("elu"))
               
model2.add(keras.layers.Dense(10 , activation = "softmax"))
               
model2.compile(loss = "sparse_categorical_crossentropy" , optimizer = "nadam" , metrics=["accuracy"])               

training the DNN, we got 1.3837 val_loss and 0.5299 val accuracy after 16 epochs, so the model is performing much better and faster after using batch normalization.

In [13]:
history2 = model2.fit(X_train,y_train,validation_data=(X_val,y_val) , batch_size = 64,epochs = 50 , 
                     callbacks=[early_stop])

Epoch 1/50
625/625 [==============================] - 6s 10ms/step - loss: 1.7932 - accuracy: 0.3579 - val_loss: 1.7142 - val_accuracy: 0.3974
Epoch 2/50
625/625 [==============================] - 6s 9ms/step - loss: 1.6039 - accuracy: 0.4273 - val_loss: 1.5725 - val_accuracy: 0.4383
Epoch 3/50
625/625 [==============================] - 6s 9ms/step - loss: 1.5275 - accuracy: 0.4564 - val_loss: 1.5661 - val_accuracy: 0.4544
Epoch 4/50
625/625 [==============================] - 6s 10ms/step - loss: 1.4670 - accuracy: 0.4769 - val_loss: 1.5230 - val_accuracy: 0.4614
Epoch 5/50
625/625 [==============================] - 6s 9ms/step - loss: 1.4210 - accuracy: 0.4911 - val_loss: 1.4772 - val_accuracy: 0.4805
Epoch 6/50
625/625 [==============================] - 6s 9ms/step - loss: 1.3781 - accuracy: 0.5112 - val_loss: 1.4484 - val_accuracy: 0.4903
Epoch 7/50
625/625 [==============================] - 6s 9ms/step - loss: 1.3410 - accuracy: 0.5225 - val_loss: 1.4194 - val_accuracy: 0.5072
Epoc

this time we will use self-normalize DNN, so we standardize the inputs, we use lecun initialization and SELU activation function (and the DNN is sequential).

In [14]:
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)
X_train_stand = (X_train - X_mean) / X_std
X_val_stand= (X_val - X_mean) / X_std
X_test_stand = (X_test - X_mean) / X_std

In [15]:
model3 = keras.models.Sequential([
    keras.layers.Flatten(input_shape = (32,32,3)) ,
])
for layer in range(20):
    model3.add(keras.layers.Dense(100 , kernel_initializer = "lecun_normal" , activation = "selu"))
               
model3.add(keras.layers.Dense(10 , activation = "softmax"))
               
model3.compile(loss = "sparse_categorical_crossentropy" , optimizer = "nadam" , metrics=["accuracy"])               

training the DNN, we got 1.4888 val_loss and 0.4906 val accuracy after 14 epcohs, so this DNN performing rather similary to the DNN with batch normalization.

In [16]:
history3 = model3.fit(X_train_stand , y_train , validation_data=(X_val_stand , y_val) , batch_size = 64 , epochs = 50,
                     callbacks=[early_stop])

Epoch 1/50
625/625 [==============================] - 4s 7ms/step - loss: 1.9459 - accuracy: 0.3033 - val_loss: 1.7762 - val_accuracy: 0.3630
Epoch 2/50
625/625 [==============================] - 4s 7ms/step - loss: 1.7292 - accuracy: 0.3821 - val_loss: 1.7289 - val_accuracy: 0.3858
Epoch 3/50
625/625 [==============================] - 4s 7ms/step - loss: 1.6252 - accuracy: 0.4245 - val_loss: 1.6940 - val_accuracy: 0.3897
Epoch 4/50
625/625 [==============================] - 4s 7ms/step - loss: 1.5518 - accuracy: 0.4544 - val_loss: 1.6360 - val_accuracy: 0.4238
Epoch 5/50
625/625 [==============================] - 4s 7ms/step - loss: 1.4975 - accuracy: 0.4740 - val_loss: 1.5687 - val_accuracy: 0.4496
Epoch 6/50
625/625 [==============================] - 4s 7ms/step - loss: 1.4480 - accuracy: 0.4915 - val_loss: 1.5656 - val_accuracy: 0.4590
Epoch 7/50
625/625 [==============================] - 4s 7ms/step - loss: 1.3999 - accuracy: 0.5125 - val_loss: 1.5920 - val_accuracy: 0.4484
Epoch 

adding to the previous DNN Alphadropout layers with ratio of 0.1 after Dense layers (Alphadropout works in self-normalize networks).

In [21]:
model4 = keras.models.Sequential([
    keras.layers.Flatten(input_shape = (32,32,3)) ,
])
for layer in range(20):
    model4.add(keras.layers.Dense(100 , kernel_initializer = "lecun_normal" , activation = "selu"))
    model4.add(keras.layers.AlphaDropout(0.1,seed=42))
               
model4.add(keras.layers.Dense(10 , activation = "softmax"))
               
model4.compile(loss = "sparse_categorical_crossentropy" , optimizer = "nadam" , metrics=["accuracy"])   

training the DNN, we got 1.9014 val_loss and 0.3631 val accuracy after 5 epochs, the model converges much faster when using rather small early stopping patience, but with worst results.

In [22]:
history4 = model4.fit(X_train_stand , y_train , validation_data=(X_val_stand , y_val) , batch_size = 64 , epochs = 50,
                     callbacks=[early_stop])

Epoch 1/50
625/625 [==============================] - 5s 7ms/step - loss: 2.1529 - accuracy: 0.2088 - val_loss: 2.1552 - val_accuracy: 0.2794
Epoch 2/50
625/625 [==============================] - 4s 7ms/step - loss: 1.9100 - accuracy: 0.2782 - val_loss: 2.0083 - val_accuracy: 0.3167
Epoch 3/50
625/625 [==============================] - 4s 7ms/step - loss: 1.8396 - accuracy: 0.3147 - val_loss: 2.0315 - val_accuracy: 0.3256
Epoch 4/50
625/625 [==============================] - 4s 7ms/step - loss: 1.7819 - accuracy: 0.3405 - val_loss: 1.9577 - val_accuracy: 0.3696
Epoch 5/50
625/625 [==============================] - 4s 7ms/step - loss: 1.7492 - accuracy: 0.3577 - val_loss: 1.9014 - val_accuracy: 0.3631
Epoch 6/50
625/625 [==============================] - 4s 7ms/step - loss: 1.7146 - accuracy: 0.3724 - val_loss: 2.0499 - val_accuracy: 0.3825
Epoch 7/50
625/625 [==============================] - 4s 7ms/step - loss: 1.6975 - accuracy: 0.3819 - val_loss: 1.9603 - val_accuracy: 0.3586
Epoch 